# 트럼프 말투로 글 작성

In [1]:
import os, re
import numpy as np
import tensorflow as tf
# path = '/aiffel/data/lyrics/'
# file_list = os.listdir(path)
# file_list_txt = [file for file in file_list if file.endswith('.txt')]

file_path = os.getenv("HOME")+'/aiffel/lyricist/data/shakespeare.txt'
with open(file_path, 'r') as f:
    raw_corpus = f.read().splitlines()
    
print(raw_corpus[:9])

['First Citizen:', 'Before we proceed any further, hear me speak.', '', 'All:', 'Speak, speak.', '', 'First Citizen:', 'You are all resolved rather to die than to famish?', '']


In [2]:
for idx, sentence in enumerate(raw_corpus):
    if len(sentence) == 0: # sentence문장이 공백일때 제외
        continue
    if sentence[-1]==":": # sentence 문장의 맨끝이 ":"일때 제외
        continue
        
    if idx>10: # 우선 문장 10개만 확인
        break
        
    print(sentence)

Before we proceed any further, hear me speak.
Speak, speak.
You are all resolved rather to die than to famish?
Resolved. resolved.


In [3]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 1
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 2
    sentence = re.sub(r'[" "]+', " ", sentence) # 3
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # 4
    sentence = sentence.strip() # 5
    sentence = '<start> ' + sentence + ' <end>' # 6
    return sentence

In [4]:
corpus = [] # 정제된 문장 넣는 빈 리스트

for sentence in raw_corpus:
    # 원치않는 문장 건너뛰기
    if len(sentence)==0: continue # 공백이 있으면 제거
    if sentence[-1]==":": continue # 각 문장 끝에 ':'이 있으면 제거
        
    # 정제하고 corpus에 넣기
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)
    
corpus[:1]

['<start> before we proceed any further , hear me speak . <end>']

In [5]:
# 토큰화
def tokenize(corpus):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        # 7000단어를 기억할 수 있는 tokenizer
        # 이미 문장을 정제하였으니 filters가 필요없다.
        # 7000 단어에 포함되지 못한 단어는 '<unk>'로 변환
        num_words=7000,
        filters='',
        oov_token='<unk>'
    )
    # corpus를 이용하여 tokenizer 내부의 단어장 완성
    tokenizer.fit_on_texts(corpus)
    # tokenizer 이용하여 corpus를 Tensor로 변환
    tensor = tokenizer.texts_to_sequences(corpus)
    # 시퀀스 길이 일정하게
    # 시퀀스가 짧으면 문장 뒤에 패딩을 붙여 길이를 맞춰준다.
    # 문장 앞에 패딩을 붙여 길이를 맞추려면 padding='pre'사용
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')
    
    
    print(tensor, tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[   2  143   40 ...    0    0    0]
 [   2  110    4 ...    0    0    0]
 [   2   11   50 ...    0    0    0]
 ...
 [   2  149 4553 ...    0    0    0]
 [   2   34   71 ...    0    0    0]
 [   2  945   34 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7ff20cd68390>


In [7]:
print(tensor[:3, :20])

[[   2  143   40  933  140  591    4  124   24  110    5    3    0    0
     0    0    0    0    0    0]
 [   2  110    4  110    5    3    0    0    0    0    0    0    0    0
     0    0    0    0    0    0]
 [   2   11   50   43 1201  316    9  201   74    9 3034   15    3    0
     0    0    0    0    0    0]]


In [8]:
for idx in tokenizer.index_word:
    print(idx,":",tokenizer.index_word[idx]) # tokenizer변수에 저장되어 있는 단어 사전의 인덱스
    
    if idx >=30:break
        
print(len(tokenizer.index_word))

1 : <unk>
2 : <start>
3 : <end>
4 : ,
5 : .
6 : the
7 : and
8 : i
9 : to
10 : of
11 : you
12 : my
13 : a
14 : that
15 : ?
16 : in
17 : !
18 : is
19 : not
20 : for
21 : s
22 : with
23 : it
24 : me
25 : your
26 : be
27 : he
28 : his
29 : but
30 : this
11179


In [18]:
# tensor에서 마지막 토큰을 잘라내서 소스문장 생성
# 마지막 토큰은 <end>가 아니고 <pad>일 가능성높음
src_input = tensor[:,:-1]
# tensor에서 <start>를 잘라내서 타겟문장 생성
tgt_input = tensor[:,1:]

print(src_input[0])
print(tgt_input[0])

[  2 931 243 931 243   3   0   0   0   0   0   0   0   0   0   0   0   0
   0   0]
[931 243 931 243   3   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0]


In [19]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input)//BATCH_SIZE

# tokenizer가 구축한 단어사전 내 7000개와, 여기 포함되지 않은 0:<pad>를 포함하여 7001개
VOCAB_SIZE = tokenizer.num_words+1

dataset=tf.data.Dataset.from_tensor_slices((src_input, tgt_input)) # tensor --> tf.data.Dataset으로 변환
dataset=dataset.shuffle(BUFFER_SIZE) # 완벽한 셔플링을 위해서는 데이터셋의 전체 크기보다 크거나 같은 Buffersize필요
dataset=dataset.batch(BATCH_SIZE,drop_remainder=True) # 256개씩 묶고 나머지 제거
dataset

<BatchDataset shapes: ((256, 20), (256, 20)), types: (tf.int32, tf.int32)>

In [20]:
class TextGenerator(tf.keras.Model):
    def __init__(self,vocab_size, embedding_size,hidden_size):
        super().__init__()
        # embedding Layer
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)# 인덱스 값을 해당 인덱스 번째의 워드벡터로 바꿔준다. embedding_size : 단어가 추상적으로 표현되는 크기
        # 2개의 LSTM Layer
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True) # return_sequences : many to many
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True) 
        # 1개의 Dense Layer
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size=256
hidden_size=1024
model = TextGenerator(tokenizer.num_words+1, embedding_size, hidden_size)

In [21]:
for src_sample, tgt_sample in dataset.take(1):break # 1개의 배치를 가져온다. 256개의 문장데이터
    
model(src_sample)

<tf.Tensor: shape=(256, 20, 7001), dtype=float32, numpy=
array([[[-1.67167760e-04, -1.38021815e-05,  3.20352236e-04, ...,
          7.84341901e-05, -4.04167513e-04, -5.66555063e-05],
        [-1.19415607e-04, -1.62487180e-04,  7.36956601e-04, ...,
          6.40187587e-04, -5.34172053e-04,  3.05261965e-05],
        [-4.75398672e-04, -2.02538693e-04,  1.14611280e-03, ...,
          8.02769966e-04, -4.64282930e-04, -2.05434244e-05],
        ...,
        [-4.12330191e-05,  2.20114216e-05, -9.62804130e-04, ...,
          7.08130479e-04, -3.28045990e-03,  1.06275373e-03],
        [-1.38052143e-04,  7.34544446e-05, -1.02782028e-03, ...,
          6.20121893e-04, -3.38425976e-03,  9.61546262e-04],
        [-2.38316396e-04,  1.21572812e-04, -1.08322571e-03, ...,
          5.31307189e-04, -3.46683944e-03,  8.62374087e-04]],

       [[-1.67167760e-04, -1.38021815e-05,  3.20352236e-04, ...,
          7.84341901e-05, -4.04167513e-04, -5.66555063e-05],
        [-3.21891566e-04,  1.27901592e-06,  8.

In [22]:
model.summary()

Model: "text_generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  1792256   
_________________________________________________________________
lstm (LSTM)                  multiple                  5246976   
_________________________________________________________________
lstm_1 (LSTM)                multiple                  8392704   
_________________________________________________________________
dense (Dense)                multiple                  7176025   
Total params: 22,607,961
Trainable params: 22,607,961
Non-trainable params: 0
_________________________________________________________________


In [27]:
optimizer = tf.keras.optimizers.Adam() 
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset, epochs=30)

Epoch 1/30
24/24 [==============================] - 11s 365ms/step - loss: 2.8181
Epoch 2/30
24/24 [==============================] - 9s 366ms/step - loss: 2.6428
Epoch 3/30
24/24 [==============================] - 9s 368ms/step - loss: 2.5909
Epoch 4/30
24/24 [==============================] - 9s 369ms/step - loss: 2.5687
Epoch 5/30
24/24 [==============================] - 9s 370ms/step - loss: 2.5346
Epoch 6/30
24/24 [==============================] - 9s 370ms/step - loss: 2.5119
Epoch 7/30
24/24 [==============================] - 9s 371ms/step - loss: 2.4657
Epoch 8/30
24/24 [==============================] - 9s 372ms/step - loss: 2.4203
Epoch 9/30
24/24 [==============================] - 9s 372ms/step - loss: 2.3832
Epoch 10/30
24/24 [==============================] - 9s 374ms/step - loss: 2.3291
Epoch 11/30
24/24 [==============================] - 9s 373ms/step - loss: 2.2765
Epoch 12/30
24/24 [==============================] - 9s 373ms/step - loss: 2.2346
Epoch 13/30
24/24 [=====

In [28]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환합니다
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나씩 예측해 문장을 만듭니다
    #    1. 입력받은 문장의 텐서를 입력합니다
    #    2. 예측된 값 중 가장 높은 확률인 word index를 뽑아냅니다
    #    3. 2에서 예측된 word index를 문장 뒤에 붙입니다
    #    4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마칩니다
    while True:
        # 1
        predict = model(test_tensor) 
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

In [29]:
generate_text(model, tokenizer, init_sentence="<start> he") ## 4-6

'<start> he said i know what you got a vacuum mouth <end> '